In [300]:
%load_ext autoreload
%autoreload 2

# Librerías para tratamiento de datos

import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np
import re

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones
from src import soporte_variables as sv

# Librería para acceder a funcionalidades del sistema operativo
import os

# Librerías para trabajar con distintos formatos de archivos
import pickle
import json

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

# BeautifulSoup y Selenium
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Consulta de tipos de contenido

In [14]:

url = "https://moviesdatabase.p.rapidapi.com/titles/utils/titleTypes"

headers = {
	"x-rapidapi-key": "e9d53ce8f2msh50c48f79aa0b1b1p1674b7jsn7a3fd4b9a409",
	"x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())

{'results': [None, 'movie', 'musicVideo', 'podcastEpisode', 'podcastSeries', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot', 'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame']}


## Consulta de títulos

Hago consulta a la API

In [295]:
resultados_peliculas = sf.consulta_peliculas(1)

Procesando géneros: 100%|██████████| 8/8 [20:11<00:00, 151.44s/it]


Compruebo longitud de resultados (tuplas)

In [296]:
len(resultados_peliculas)

596

Almaceno resultados

In [299]:
with open('datos/resultados_peliculas.pkl', 'wb') as f:
    pickle.dump(resultados_peliculas, f)

Creo lista con IDs

In [313]:
lista_ids = []

for resultado in resultados_peliculas:
    lista_ids.append(resultado[0])

In [315]:
len(lista_ids)

596

A través de s

In [316]:
def scrap_imdb(lista_ids):

    # Configurar el driver y abrir la URL
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(options=chrome_options)
    
    url_inicio = f"https://www.imdb.com/"
    driver.get(url_inicio)
    
    # Esperar a que aparezca el banner de cookies y aceptar cookies
    time.sleep(3)
    try:
        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        print("Cookies aceptadas")
    except Exception as e:
        print(f"Error al aceptar cookies: {e}")

    lista_sopas = []
    time.sleep(2)
    
    # Iterar sobre las páginas
    for id in tqdm(lista_ids):
        # Navegar a la página de resultados
        pagina_url = f"https://www.imdb.com/title/{id}"
        driver.get(pagina_url)
        print(f"Scraping página {id}")

        # Obtener el código fuente de la página y analizarlo con BeautifulSoup
        time.sleep(2)

        try:
            source = driver.page_source
            sopa = BeautifulSoup(source, "html.parser")
            print(f"Sopa creada {id}")
            time.sleep(1)

            calificacion = sopa.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div.sc-3a4309f8-0.jJkxPn.sc-70a366cc-1.kUfGfN > div > div:nth-child(1) > a > span > div > div.sc-d541859f-0.hNIoIx > div.sc-d541859f-2.kxphVf > span.sc-d541859f-1.imUuxf')
            direccion = sopa.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eZLbYw > div.sc-9a2a0028-6.iMFgzR > div.sc-9a2a0028-10.dFokEJ > section > div.sc-70a366cc-3.iwmAOx > div > ul > li:nth-child(1) > div')
            guionista = sopa.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eZLbYw > div.sc-9a2a0028-6.iMFgzR > div.sc-9a2a0028-10.dFokEJ > section > div.sc-70a366cc-3.iwmAOx > div > ul > li:nth-child(2) > div')
            argumento = sopa.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eZLbYw > div.sc-9a2a0028-6.iMFgzR > div.sc-9a2a0028-10.dFokEJ > section > p > span.sc-3ac15c8d-1.gkeSEi')        
            duracion = sopa.select_one('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(2)')

            calificacion = calificacion.text.strip()
            direccion = direccion.text.strip()
            guionista = guionista.text.strip()
            argumento = argumento.text.strip()
            duracion = duracion.text.strip()

            # Almacenar resultados de la página
            lista_sopas.append((id, calificacion, direccion, guionista, argumento, duracion))
        
        except:
            
    
    # Cerrar el navegador
    time.sleep(2)
    driver.quit()
    
    # Imprimir confirmación
    print("Scraping finalizado")
    
    return lista_sopas

In [318]:
sopas = scrap_imdb(lista_ids)

Cookies aceptadas


  0%|          | 0/596 [00:00<?, ?it/s]

Scraping página tt0059325
Sopa creada tt0059325


  0%|          | 1/596 [00:04<42:46,  4.31s/it]

Scraping página tt0059900
Sopa creada tt0059900


...

Scraping página tt0096845
Sopa creada tt0096845


  4%|▍         | 23/596 [02:04<46:15,  4.84s/it]

Scraping página tt0097339
Sopa creada tt0097339


  4%|▍         | 23/596 [02:09<53:47,  5.63s/it]


AttributeError: 'NoneType' object has no attribute 'text'